# Continuous Deployment and Versioning

If we've followed the steps so far, we're pretty confident that the code in our `master` branch works, and always represents the latest and greatest version of our package. At certain points of time, we will want to *release* (or *deploy*) new versions of our code.


*Releasing* the code in this context means that we will assign a new version number to our package and make it easily accessible to our users - either by uploading the new version on PyPI, or creating an easily downloadable `.zip` file, or both - anything that doesn't involve `git clone`ing our repository.

This release process may happen as frequently as we wish - even several times a day in the case of fixing bugs in our package. As long as we follow [Semantic Versioning](https://devhints.io/semver) for our package, everything downstream should work just fine (Our users could do a `pip install walrus==3.*` and expect to get the latest 3.x release, for example).


![semver](https://bytearcher.com/goodies/semantic-versioning-cheatsheet/wheelbarrel-no-tilde-caret-white-bg-w1000.jpg)

(Image from https://bytearcher.com/goodies/semantic-versioning-cheatsheet/)

## A Single Version number for our package

Earlier, we noticed that we had to specify the version number of the `walrus` package at two separate places:
    
```
.
├── dist
├── pyproject.toml
├── README.md
├── setup.cfg                 <--- Version Number specified as packaging metadata
├── src
│   └── walrus
│       ├── __init__.py       <--- Version Number specified as __version__
│       └── math.py
└── test
    ├── test_basic.py
    └── test_math.py

```

There are a few ways to make version numbers at several places consistent:

 - Manually ensure that version numbers are the same, wherever they occur. This is of course tedious and error prone.
 - Use a tool like [bumpversion](https://github.com/peritus/bumpversion).
     - We create a configuration file that tells `bumpversion` exactly what files to tweak whenever we want to bump the version.
     - We issue a command like `bumpversion minor` to bump up the minor version number, or `bumpversion 0.6.3` to specify the complete version number, and it takes care of the rest.
 - Use something like `setuptools-scm` to get the version number directly from the **git tag**.
 
 
 In our opinion, `setuptools-scm` is the best way, since it takes care of a few scenarios:
 
 - We only have to specify the version once (of course)
 - As long as we're on `git`, there's never a confusion as to the current state of the code w.r.t. the version (Did we make changes after bumping the version? Did we remember to bump the version after changing our code in `git`?)
 - As long as we set this up correctly (as we'll see soon), no two developers in our project will end up using the same version number. The version number will be assigned centrally, and *atomically*.
 
 ### TLDR; - In the `setuptools-scm` world, the *git tag* **IS** the version number.

### Example

The folder `04-walrus-package/` is the next iteration of our code. The only changes here are in `pyproject.toml`:
    
```
[build-system]
requires = [
    "setuptools>=42",
    "wheel",
    "setuptools_scm>=6.2"
]
build-backend = "setuptools.build_meta"


[tool.setuptools_scm]
write_to = "src/walrus/_version.py"
```
    
We're telling `pip` that we need the `setuptools_scm` during the build phase of our package, and we're telling `setuptools_scm` which file to write the version number to. In this case, `src/walrus/_version.py`. *Note that we will never modify this file manually, nor add it to version control.*

Let's see how this works on the command line..

### Continuous Deployment on GitHub

#### What's a GitHub 'release'?

GitHub has a feature to create a [release](https://github.com/vineetbansal/walrus/releases). This is essentially doing these things:
  - Add a `git tag` to a branch.
  - Zip up the current state of the branch for archival purposes.
  - Allow us to add a title + description to this release.
  - Allow us to use `Github Actions` to do anything else that we want..
  
  We will be using this last bullet point above to manage our package releases on PyPI.
  
The repository [walrus](https://github.com/vineetbansal/walrus) on GitHub has a [release workflow file](https://github.com/vineetbansal/walrus/blob/master/.github/workflows/release.yml) that we can inspect. Note that:

  - The `build_sdist` and the `build_wheel` jobs are hopefully self-explanatory by now, and generate the `sdist` and `wheel` files in the `dist/` folder. Note that we generate the `sdist` only once, but the `wheel` for as many platforms as we purport to support. In this case, it doesn't really matter since our package is pure python. For packages that have a C/C++ component, like [this one](https://github.com/vineetbansal/walrus_carpenter), the `build_wheel` job generates a whole bunch of wheel files, each with a unique name designating the platform + python version.


  - An `upload_all` job is executed once the `build_sdist` and `build_wheel` jobs are complete
  
  ```
  upload_all:
    name: Upload if release
    needs: [build_sdist, build_wheels]
    runs-on: ubuntu-latest
    if: github.event_name == 'release' && github.event.action == 'published'

    steps:
    - uses: actions/download-artifact@v2
      with:
        name: artifact
        path: dist

    - uses: pypa/gh-action-pypi-publish@release/v1
      with:
        user: __token__
        password: ${{ secrets.PYPI_API_TOKEN }}
   ```
   
   This job is triggered when we use the GitHub UI to create a release, and is using:
   
   - A marketplace action [download-artifact](https://github.com/actions/download-artifact) to collect everything in the `dist/` folder and make it available as a `artifact.zip` file.
     
   - A marketplace action [gh-action-pypi-publish](https://github.com/pypa/gh-action-pypi-publish) to push eveything in the `dist/` folder to our PyPI account. (This step requires us to share a secret token from our PyPI account to our GitHub repository).